In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import re
import imp
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q cltk==1.0.22

Note: you may need to restart the kernel to use updated packages.


In [4]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment

imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)
imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)

<module 'p_aligment.p_aligment' from '../src/p_aligment/p_aligment.py'>

# shared vocabulary suffix research

In [9]:
london_breslau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/london_breslau.csv').drop(['Unnamed: 0'], axis=1)
london_zwickau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/london_zwickau.csv').drop(['Unnamed: 0'], axis=1)
zwickau_breslau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/zwickau_breslau.csv').drop(['Unnamed: 0'], axis=1)

In [12]:
london_zwickau_shared_words

,shared_word,version a name,version a count,version b name,version b count
0,iacere,london,19,zwickau,14
1,longum,london,15,zwickau,10
2,oppidum,london,15,zwickau,1
3,ueni,london,15,zwickau,10
4,campo,london,13,zwickau,7
...,...,...,...,...,...
2794,terrere,london,1,zwickau,1
2795,margat,london,1,zwickau,2
2796,ualaniam,london,1,zwickau,1
2797,doram,london,1,zwickau,1


In [34]:
def count_suffix(corpus, suffix):
    result = {}
    
    for i, p in enumerate(corpus):
        matches = 0
        
        for w in p.split():
            if w.endswith(suffix): matches += 1
                
        if matches > 0:
            result[i] = {'suffix': suffix, 'p_index': i, 'matches': matches}
    
    return result

In [36]:
for index, row in london_breslau_shared_words.iterrows():
    suffix = row['shared_word'][-4:]
    result = {}

    resp = count_suffix(thesisDataReader.get_london_by_new_line(), suffix)
    if resp is not None: 
        print(resp)
        print()

{8: {'suffix': 'tiim', 'p_index': 8, 'matches': 1}, 13: {'suffix': 'tiim', 'p_index': 13, 'matches': 1}, 14: {'suffix': 'tiim', 'p_index': 14, 'matches': 1}, 102: {'suffix': 'tiim', 'p_index': 102, 'matches': 1}, 103: {'suffix': 'tiim', 'p_index': 103, 'matches': 1}, 104: {'suffix': 'tiim', 'p_index': 104, 'matches': 1}, 107: {'suffix': 'tiim', 'p_index': 107, 'matches': 1}, 196: {'suffix': 'tiim', 'p_index': 196, 'matches': 1}, 212: {'suffix': 'tiim', 'p_index': 212, 'matches': 1}, 230: {'suffix': 'tiim', 'p_index': 230, 'matches': 1}, 247: {'suffix': 'tiim', 'p_index': 247, 'matches': 1}}

{17: {'suffix': 'inde', 'p_index': 17, 'matches': 1}, 21: {'suffix': 'inde', 'p_index': 21, 'matches': 1}, 24: {'suffix': 'inde', 'p_index': 24, 'matches': 3}, 25: {'suffix': 'inde', 'p_index': 25, 'matches': 4}, 31: {'suffix': 'inde', 'p_index': 31, 'matches': 2}, 36: {'suffix': 'inde', 'p_index': 36, 'matches': 1}, 42: {'suffix': 'inde', 'p_index': 42, 'matches': 1}, 48: {'suffix': 'inde', 'p_ind

{138: {'suffix': 'bter', 'p_index': 138, 'matches': 1}, 169: {'suffix': 'bter', 'p_index': 169, 'matches': 1}, 205: {'suffix': 'bter', 'p_index': 205, 'matches': 1}}

{196: {'suffix': 'paim', 'p_index': 196, 'matches': 1}, 225: {'suffix': 'paim', 'p_index': 225, 'matches': 1}, 260: {'suffix': 'paim', 'p_index': 260, 'matches': 1}}

{1: {'suffix': 'ubin', 'p_index': 1, 'matches': 1}, 287: {'suffix': 'ubin', 'p_index': 287, 'matches': 1}}

{1: {'suffix': 'enti', 'p_index': 1, 'matches': 1}, 4: {'suffix': 'enti', 'p_index': 4, 'matches': 1}, 20: {'suffix': 'enti', 'p_index': 20, 'matches': 3}, 28: {'suffix': 'enti', 'p_index': 28, 'matches': 1}, 52: {'suffix': 'enti', 'p_index': 52, 'matches': 1}, 71: {'suffix': 'enti', 'p_index': 71, 'matches': 1}, 92: {'suffix': 'enti', 'p_index': 92, 'matches': 1}, 94: {'suffix': 'enti', 'p_index': 94, 'matches': 1}, 150: {'suffix': 'enti', 'p_index': 150, 'matches': 1}, 155: {'suffix': 'enti', 'p_index': 155, 'matches': 1}, 157: {'suffix': 'enti', 'p_

{116: {'suffix': 'onta', 'p_index': 116, 'matches': 1}, 120: {'suffix': 'onta', 'p_index': 120, 'matches': 1}}

{1: {'suffix': 'olis', 'p_index': 1, 'matches': 1}, 11: {'suffix': 'olis', 'p_index': 11, 'matches': 1}, 12: {'suffix': 'olis', 'p_index': 12, 'matches': 1}, 13: {'suffix': 'olis', 'p_index': 13, 'matches': 1}, 14: {'suffix': 'olis', 'p_index': 14, 'matches': 1}, 15: {'suffix': 'olis', 'p_index': 15, 'matches': 2}, 16: {'suffix': 'olis', 'p_index': 16, 'matches': 1}, 17: {'suffix': 'olis', 'p_index': 17, 'matches': 2}, 18: {'suffix': 'olis', 'p_index': 18, 'matches': 1}, 25: {'suffix': 'olis', 'p_index': 25, 'matches': 1}, 26: {'suffix': 'olis', 'p_index': 26, 'matches': 1}, 27: {'suffix': 'olis', 'p_index': 27, 'matches': 2}, 34: {'suffix': 'olis', 'p_index': 34, 'matches': 1}, 37: {'suffix': 'olis', 'p_index': 37, 'matches': 1}, 71: {'suffix': 'olis', 'p_index': 71, 'matches': 1}, 88: {'suffix': 'olis', 'p_index': 88, 'matches': 2}, 92: {'suffix': 'olis', 'p_index': 92, 'ma

{1: {'suffix': 'ntem', 'p_index': 1, 'matches': 2}, 2: {'suffix': 'ntem', 'p_index': 2, 'matches': 3}, 3: {'suffix': 'ntem', 'p_index': 3, 'matches': 12}, 5: {'suffix': 'ntem', 'p_index': 5, 'matches': 6}, 17: {'suffix': 'ntem', 'p_index': 17, 'matches': 1}, 18: {'suffix': 'ntem', 'p_index': 18, 'matches': 1}, 21: {'suffix': 'ntem', 'p_index': 21, 'matches': 2}, 26: {'suffix': 'ntem', 'p_index': 26, 'matches': 1}, 32: {'suffix': 'ntem', 'p_index': 32, 'matches': 1}, 38: {'suffix': 'ntem', 'p_index': 38, 'matches': 2}, 41: {'suffix': 'ntem', 'p_index': 41, 'matches': 1}, 45: {'suffix': 'ntem', 'p_index': 45, 'matches': 1}, 49: {'suffix': 'ntem', 'p_index': 49, 'matches': 2}, 51: {'suffix': 'ntem', 'p_index': 51, 'matches': 1}, 53: {'suffix': 'ntem', 'p_index': 53, 'matches': 2}, 54: {'suffix': 'ntem', 'p_index': 54, 'matches': 1}, 60: {'suffix': 'ntem', 'p_index': 60, 'matches': 2}, 63: {'suffix': 'ntem', 'p_index': 63, 'matches': 2}, 65: {'suffix': 'ntem', 'p_index': 65, 'matches': 1},

{11: {'suffix': 'olim', 'p_index': 11, 'matches': 1}, 16: {'suffix': 'olim', 'p_index': 16, 'matches': 1}, 17: {'suffix': 'olim', 'p_index': 17, 'matches': 1}, 18: {'suffix': 'olim', 'p_index': 18, 'matches': 1}, 32: {'suffix': 'olim', 'p_index': 32, 'matches': 2}, 38: {'suffix': 'olim', 'p_index': 38, 'matches': 1}, 80: {'suffix': 'olim', 'p_index': 80, 'matches': 2}, 126: {'suffix': 'olim', 'p_index': 126, 'matches': 1}, 144: {'suffix': 'olim', 'p_index': 144, 'matches': 1}, 150: {'suffix': 'olim', 'p_index': 150, 'matches': 1}, 188: {'suffix': 'olim', 'p_index': 188, 'matches': 1}, 212: {'suffix': 'olim', 'p_index': 212, 'matches': 1}, 239: {'suffix': 'olim', 'p_index': 239, 'matches': 1}, 253: {'suffix': 'olim', 'p_index': 253, 'matches': 1}, 254: {'suffix': 'olim', 'p_index': 254, 'matches': 1}, 273: {'suffix': 'olim', 'p_index': 273, 'matches': 2}, 290: {'suffix': 'olim', 'p_index': 290, 'matches': 1}, 294: {'suffix': 'olim', 'p_index': 294, 'matches': 1}, 304: {'suffix': 'olim',

...

{3: {'suffix': 'atis', 'p_index': 3, 'matches': 1}, 8: {'suffix': 'atis', 'p_index': 8, 'matches': 1}, 23: {'suffix': 'atis', 'p_index': 23, 'matches': 3}, 24: {'suffix': 'atis', 'p_index': 24, 'matches': 1}, 25: {'suffix': 'atis', 'p_index': 25, 'matches': 3}, 26: {'suffix': 'atis', 'p_index': 26, 'matches': 3}, 31: {'suffix': 'atis', 'p_index': 31, 'matches': 3}, 33: {'suffix': 'atis', 'p_index': 33, 'matches': 1}, 35: {'suffix': 'atis', 'p_index': 35, 'matches': 1}, 39: {'suffix': 'atis', 'p_index': 39, 'matches': 1}, 49: {'suffix': 'atis', 'p_index': 49, 'matches': 1}, 50: {'suffix': 'atis', 'p_index': 50, 'matches': 1}, 51: {'suffix': 'atis', 'p_index': 51, 'matches': 2}, 59: {'suffix': 'atis', 'p_index': 59, 'matches': 1}, 66: {'suffix': 'atis', 'p_index': 66, 'matches': 1}, 69: {'suffix': 'atis', 'p_index': 69, 'matches': 1}, 83: {'suffix': 'atis', 'p_index': 83, 'matches': 2}, 92: {'suffix': 'atis', 'p_index': 92, 'matches': 3}, 100: {'suffix': 'atis', 'p_index': 100, 'matches'

{3: {'suffix': 'onem', 'p_index': 3, 'matches': 1}, 4: {'suffix': 'onem', 'p_index': 4, 'matches': 1}, 5: {'suffix': 'onem', 'p_index': 5, 'matches': 1}, 9: {'suffix': 'onem', 'p_index': 9, 'matches': 1}, 19: {'suffix': 'onem', 'p_index': 19, 'matches': 2}, 20: {'suffix': 'onem', 'p_index': 20, 'matches': 1}, 24: {'suffix': 'onem', 'p_index': 24, 'matches': 1}, 29: {'suffix': 'onem', 'p_index': 29, 'matches': 1}, 31: {'suffix': 'onem', 'p_index': 31, 'matches': 1}, 33: {'suffix': 'onem', 'p_index': 33, 'matches': 1}, 38: {'suffix': 'onem', 'p_index': 38, 'matches': 1}, 40: {'suffix': 'onem', 'p_index': 40, 'matches': 2}, 44: {'suffix': 'onem', 'p_index': 44, 'matches': 2}, 47: {'suffix': 'onem', 'p_index': 47, 'matches': 2}, 50: {'suffix': 'onem', 'p_index': 50, 'matches': 2}, 52: {'suffix': 'onem', 'p_index': 52, 'matches': 2}, 54: {'suffix': 'onem', 'p_index': 54, 'matches': 1}, 55: {'suffix': 'onem', 'p_index': 55, 'matches': 1}, 59: {'suffix': 'onem', 'p_index': 59, 'matches': 2}, 

{2: {'suffix': 'tate', 'p_index': 2, 'matches': 1}, 3: {'suffix': 'tate', 'p_index': 3, 'matches': 1}, 16: {'suffix': 'tate', 'p_index': 16, 'matches': 1}, 20: {'suffix': 'tate', 'p_index': 20, 'matches': 1}, 21: {'suffix': 'tate', 'p_index': 21, 'matches': 1}, 25: {'suffix': 'tate', 'p_index': 25, 'matches': 1}, 26: {'suffix': 'tate', 'p_index': 26, 'matches': 1}, 27: {'suffix': 'tate', 'p_index': 27, 'matches': 2}, 28: {'suffix': 'tate', 'p_index': 28, 'matches': 2}, 31: {'suffix': 'tate', 'p_index': 31, 'matches': 1}, 33: {'suffix': 'tate', 'p_index': 33, 'matches': 1}, 34: {'suffix': 'tate', 'p_index': 34, 'matches': 1}, 37: {'suffix': 'tate', 'p_index': 37, 'matches': 1}, 38: {'suffix': 'tate', 'p_index': 38, 'matches': 1}, 46: {'suffix': 'tate', 'p_index': 46, 'matches': 2}, 51: {'suffix': 'tate', 'p_index': 51, 'matches': 3}, 57: {'suffix': 'tate', 'p_index': 57, 'matches': 1}, 67: {'suffix': 'tate', 'p_index': 67, 'matches': 1}, 69: {'suffix': 'tate', 'p_index': 69, 'matches': 

{1: {'suffix': 'ibus', 'p_index': 1, 'matches': 3}, 2: {'suffix': 'ibus', 'p_index': 2, 'matches': 3}, 4: {'suffix': 'ibus', 'p_index': 4, 'matches': 1}, 5: {'suffix': 'ibus', 'p_index': 5, 'matches': 5}, 6: {'suffix': 'ibus', 'p_index': 6, 'matches': 5}, 7: {'suffix': 'ibus', 'p_index': 7, 'matches': 3}, 9: {'suffix': 'ibus', 'p_index': 9, 'matches': 2}, 10: {'suffix': 'ibus', 'p_index': 10, 'matches': 1}, 13: {'suffix': 'ibus', 'p_index': 13, 'matches': 1}, 21: {'suffix': 'ibus', 'p_index': 21, 'matches': 1}, 23: {'suffix': 'ibus', 'p_index': 23, 'matches': 6}, 24: {'suffix': 'ibus', 'p_index': 24, 'matches': 3}, 25: {'suffix': 'ibus', 'p_index': 25, 'matches': 4}, 26: {'suffix': 'ibus', 'p_index': 26, 'matches': 6}, 27: {'suffix': 'ibus', 'p_index': 27, 'matches': 3}, 28: {'suffix': 'ibus', 'p_index': 28, 'matches': 1}, 31: {'suffix': 'ibus', 'p_index': 31, 'matches': 1}, 36: {'suffix': 'ibus', 'p_index': 36, 'matches': 1}, 37: {'suffix': 'ibus', 'p_index': 37, 'matches': 4}, 38: {'

{4: {'suffix': 'arum', 'p_index': 4, 'matches': 1}, 8: {'suffix': 'arum', 'p_index': 8, 'matches': 2}, 19: {'suffix': 'arum', 'p_index': 19, 'matches': 1}, 25: {'suffix': 'arum', 'p_index': 25, 'matches': 1}, 26: {'suffix': 'arum', 'p_index': 26, 'matches': 2}, 28: {'suffix': 'arum', 'p_index': 28, 'matches': 5}, 36: {'suffix': 'arum', 'p_index': 36, 'matches': 1}, 42: {'suffix': 'arum', 'p_index': 42, 'matches': 2}, 53: {'suffix': 'arum', 'p_index': 53, 'matches': 1}, 68: {'suffix': 'arum', 'p_index': 68, 'matches': 2}, 75: {'suffix': 'arum', 'p_index': 75, 'matches': 1}, 82: {'suffix': 'arum', 'p_index': 82, 'matches': 1}, 83: {'suffix': 'arum', 'p_index': 83, 'matches': 1}, 102: {'suffix': 'arum', 'p_index': 102, 'matches': 1}, 103: {'suffix': 'arum', 'p_index': 103, 'matches': 1}, 106: {'suffix': 'arum', 'p_index': 106, 'matches': 3}, 115: {'suffix': 'arum', 'p_index': 115, 'matches': 1}, 116: {'suffix': 'arum', 'p_index': 116, 'matches': 1}, 117: {'suffix': 'arum', 'p_index': 117,

In [28]:
result

{50: {'suffix': 'blin', 'p_index': 50, 'matches': 1},
 316: {'suffix': 'blin', 'p_index': 316, 'matches': 1}}